# 各種モジュールの読み込み

必要なモジュールを読み込む。


In [ ]:
from sentence_transformers import SentenceTransformer
import sklearn
from sklearn.metrics import v_measure_score
from torch import embedding
import torch
from sklearn import cluster


import json
import numpy as np

# Ex.1  Josnの読み込み

In [ ]:
import json
import codecs
import sentencepiece as spm


# load JSON 20newsgroup data
with open("newsgroups.json") as fd:
    data = json.load(fd)
# content, target(=class ID), target_name(=newsgroup name)
content = data["content"]
target = data["target"]
target_names = data["target_names"]
# get dictionary values for content, target, and target_names
content_value = content.values()
target_value = target.values()
target_name_value = target_names.values()
# extract lists for content, target, and target_names
content_value_list = list(content_value)  # メッセージテキスト本体
target_value_list = list(target_value)  # メッセージのカテゴリ ID
target_namevalue_list = list(target_name_value)
num_docs = 11314  # (=len(content.keys())


# Ex.2　文章のEmbedding
今回はSentence Transformerを用い、all-MiniLM-L6-v2を用いてembedする。

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")
vec_data: np.ndarray = model.encode(content_value_list, convert_to_numpy=True)

# Ex.3 クラスタリング
K-means法を用いてクラスタリング

In [ ]:
# assume fixed‐legth vector data is kept in "vec_data"
k_means = cluster.KMeans(n_clusters=20)
k_means.fit(vec_data)
predicted = k_means.predict(vec_data)

# Ex.4-1 
purityの値の計算

In [ ]:
def create_cross_table(truth, predict):
    result = [[0] * len(set(truth)) for _ in range(len(set(predict)))]
    for p, t in zip(predict, truth):
        result[p][t] += 1
    return result


def purity(CT):
    return np.sum(np.max(CT, axis=1)) / np.sum(CT)


CT = create_cross_table(target_value_list, predicted)
purity_val = purity(CT)
print(f"purity_val{purity_val}")

purity_val0.6439809086088033

# Ex.4-2

In [ ]:
print(type(target_name_value), type(predicted))
v_measure_val = sklearn.metrics.v_measure_score(list(target_name_value), predicted)
print(f"v_measure_val{v_measure_val}"

v_measure_val0.6164935268881546

# Ex.5 考察
PurityとV-measureから想像される 20newsgroup データを選択したクラスタリング手法のもとで、どうして、これらの値に帰着したかの分析、
ならびに、入力の 20newsgroup データ（原則 11314 文書）から、今回のプログラム課題で、どこが計算時間的にボトルネックであったかの考察も加えてくれることを期待します。




K means法は与えられたパラメータkをもとに、k個に分割するようなモデルです。すべての点を総当りして重心を計算するため、重心から近いものがクラスタリンスされる。
そのため、楕円形やその他の形の場合、クラスタリングされない場合があるなど、非常に欠点が多く、その結果が評価指標に現れていると考えられる。

TF-IDFの場合は非常に小さい値、となったがTF−IDFではクラスタリングをするためのベクトルに変換しきれていない可能性が考えられる。TF-IDFは単語ベースで計算しているので、文章全体を見ることができていない。
一方AttentionベースのTransformerである場合、すべての文章を参照してベクトルを生成するため、文章全体を見たベクトルを生成することができる。

プログラムの計算時間的のボトルネックについて、TransformerのモデルでEmbedding　しているため、パラメータが膨大なため、計算量が多くなり、実行時間は１０分〜２０分必要であり、非常に重たかった。